In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

#  Create datasets
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)

#  Create DataLoaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:13<00:00, 12.3MB/s]


In [3]:
! nvidia-smi

Thu Nov 13 12:18:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
torch.cuda.is_available()

True

In [5]:
def train(loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

def evaluate(loader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

In [6]:
import torch.nn.functional as F # Import F

class CNN_v3(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()


        self.conv1_1 = nn.Conv2d(in_channels, 128, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(128)
        self.relu1_1 = nn.ReLU()
        self.conv1_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn1_2 = nn.BatchNorm2d(128)
        self.relu1_2 = nn.ReLU()
        self.conv1_3 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn1_3 = nn.BatchNorm2d(128)
        self.relu1_3 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)



        self.conv2_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(256)
        self.relu2_1 = nn.ReLU()
        self.conv2_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn2_2 = nn.BatchNorm2d(256)
        self.relu2_2 = nn.ReLU()
        self.conv2_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn2_3 = nn.BatchNorm2d(256)
        self.relu2_3 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)



        self.conv3_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(512)
        self.relu3_1 = nn.ReLU()
        self.conv3_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn3_2 = nn.BatchNorm2d(512)
        self.relu3_2 = nn.ReLU()
        self.conv3_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn3_3 = nn.BatchNorm2d(512)
        self.relu3_3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)



        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(512, 128)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, num_classes)


    def forward(self, x):

        x = self.conv1_1(x)
        x = self.bn1_1(x)
        x = self.relu1_1(x)
        x = self.conv1_2(x)
        x = self.bn1_2(x)
        x = self.relu1_2(x)
        x = self.conv1_3(x)
        x = self.bn1_3(x)
        x = self.relu1_3(x)
        x = self.pool1(x)



        x = self.conv2_1(x)
        x = self.bn2_1(x)
        x = self.relu2_1(x)
        x = self.conv2_2(x)
        x = self.bn2_2(x)
        x = self.relu2_2(x)
        x = self.conv2_3(x)
        x = self.bn2_3(x)
        x = self.relu2_3(x)
        x = self.pool2(x)



        x = self.conv3_1(x)
        x = self.bn3_1(x)
        x = self.relu3_1(x)
        x = self.conv3_2(x)
        x = self.bn3_2(x)
        x = self.relu3_2(x)
        x = self.conv3_3(x)
        x = self.bn3_3(x)
        x = self.relu3_3(x)
        x = self.pool3(x)


        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [7]:
metrics = {
    'train' : {
        'loss' : [],
        'acc' : []
    },
    'eval' : {
        'loss' : [],
        'acc' : []
    }
}
def metrics_gatherer(train_loss, train_acc, eval_loss, eval_acc):
    metrics['train']['loss'].append(train_loss)
    metrics['train']['acc'].append(train_acc)
    metrics['eval']['loss'].append(eval_loss)
    metrics['eval']['acc'].append(eval_acc)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN_v3(in_channels=3, num_classes=100)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [11]:
epochs=200
for epoch in range(1, epochs+1):
    train_loss, train_acc = train(trainloader, model, criterion, optimizer, device)
    eval_loss, eval_acc = evaluate(testloader, model, criterion, device)
    metrics_gatherer(train_loss, train_acc, eval_loss, eval_acc)
    print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss} - Train Acc: {train_acc} // Eval Loss: {eval_loss}, Eval Acc: {eval_acc}")
    scheduler.step()

Evaluating: 100%|██████████| 100/100 [00:03<00:00, 29.99it/s]


Epoch 1/200 - Train Loss: 0.09719695443153381 - Train Acc: 0.97104 // Eval Loss: 1.798085482120514, Eval Acc: 0.6981


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 26.15it/s]


Epoch 2/200 - Train Loss: 0.09775887603759766 - Train Acc: 0.97002 // Eval Loss: 1.8121624088287354, Eval Acc: 0.7004


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 28.11it/s]


...

Epoch 198/200 - Train Loss: 0.08526794151306152 - Train Acc: 0.97458 // Eval Loss: 1.8548920404911042, Eval Acc: 0.7024


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 27.29it/s]


Epoch 199/200 - Train Loss: 0.08483538661956787 - Train Acc: 0.97498 // Eval Loss: 1.8270678794384003, Eval Acc: 0.7025


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 27.22it/s]

Epoch 200/200 - Train Loss: 0.08430538806200027 - Train Acc: 0.97498 // Eval Loss: 1.849651750922203, Eval Acc: 0.702
